## DDA3020 Autumn 2023 Homework 4

### Programming Task 2.1

#### student ID: 121090429
#### Name: Ou Ziyi

--------------------------------------------------------------------------------------------------------------------------


In [136]:
import numpy as np

np.random.seed(55)  # set seed for reproducibility

## 1. Two clustering algorithms

### (1) K-means

In [137]:
class KMeans:
    def __init__(self, n_clusters, max_iters=100):
        self.n_clusters = n_clusters
        self.max_iters = max_iters

    def fit(self, X):
        n_samples, n_features = X.shape

        # Initialize centroids randomly
        centroids = X[np.random.choice(n_samples, self.n_clusters, replace=False)]
        
        for _ in range(self.max_iters):
            # Assign each point to the nearest centroid
            labels = np.argmin(np.linalg.norm(X[:, np.newaxis] - centroids, axis=2), axis=1)
            
            # Update centroids
            new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(self.n_clusters)])
            
            # Check for convergence
            if np.all(centroids == new_centroids):
                break
            
            centroids = new_centroids

        self.centroids = centroids
        self.labels = labels


### (2) GMM

In [138]:
from scipy.stats import multivariate_normal

class GMM:
    def __init__(self, n_components, max_iters=100,allow_singular=True):
        self.n_components = n_components
        self.max_iters = max_iters
        self.weights = None
        self.means = None
        self.covariances = None
        self.labels = None  # Add labels attribute
        self.allow_singular = allow_singular

    def fit(self, X):
        n_samples, n_features = X.shape

        # Initialize parameters
        self.weights = np.ones(self.n_components) / self.n_components
        self.means = X[np.random.choice(n_samples, self.n_components, replace=False)]
        self.covariances = [np.cov(X.T) for _ in range(self.n_components)]

        for _ in range(self.max_iters):
            # Expectation step
            responsibilities = np.array([self.weights[i] * multivariate_normal.pdf(X, self.means[i], self.covariances[i]) for i in range(self.n_components)]).T
            responsibilities /= responsibilities.sum(axis=1, keepdims=True)

            # Maximization step
            Nk = responsibilities.sum(axis=0)
            self.weights = Nk / n_samples
            self.means = np.dot(responsibilities.T, X) / Nk[:, np.newaxis]
            self.covariances = [np.dot((X - self.means[i]).T, responsibilities[:, i][:, np.newaxis] * (X - self.means[i])) / Nk[i] for i in range(self.n_components)]

        # Assign labels based on the probability distribution
        self.labels = np.argmax(responsibilities, axis=1)
        return self

    def predict_proba(self, X):
        responsibilities = np.array([self.weights[i] * multivariate_normal.pdf(X, self.means[i], self.covariances[i]) for i in range(self.n_components)]).T
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)
        return responsibilities

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

## 2. Three evaluation metrics

### (1) Silhouette Coefficient

In [139]:
import numpy as np

def silhouette_coefficient(X, labels):
    n_samples = len(X)
    a = np.zeros(n_samples)
    b = np.zeros(n_samples)

    for i in range(n_samples):
        cluster_i = labels[i]
        cluster_i_points = X[labels == cluster_i]
        a[i] = np.mean(np.linalg.norm(X[i] - cluster_i_points, axis=1))

        other_clusters = [j for j in range(len(np.unique(labels))) if j != cluster_i]
        min_distances = np.zeros(len(other_clusters))

        for idx, cluster_j in enumerate(other_clusters):
            cluster_j_points = X[labels == cluster_j]
            min_distances[idx] = np.mean(np.linalg.norm(X[i] - cluster_j_points, axis=1))

        b[i] = np.min(min_distances)

    s = (b - a) / np.maximum(a, b)
    return np.mean(s)

### (2) Rand Index

In [140]:
def rand_index(true_labels, pred_labels):
    n = len(true_labels)
    a, b = 0, 0

    for i in range(n):
        for j in range(i + 1, n):
            same_true = true_labels[i] == true_labels[j]
            same_pred = pred_labels[i] == pred_labels[j]

            if same_true and same_pred:
                a += 1
            elif not same_true and not same_pred:
                b += 1

    # Rand Index = (a + b) / C(n, 2)
    return (a + b) / (n * (n - 1) / 2)

### (3) Normalized Mutual Information (NMI)

In [141]:
def NMI(true_labels, pred_labels):
    def entropy(labels):
        unique_labels, counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        return -np.sum(probabilities * np.log2(probabilities))

    # Mutual Information
    h_true = entropy(true_labels)
    h_pred = entropy(pred_labels)
    h_joint = entropy(np.vstack((true_labels, pred_labels)).T)

    mi = h_true + h_pred - h_joint

    # Normalized Mutual Information
    nmi = mi / np.sqrt(h_true * h_pred)
    return nmi

## 3. Apply clustering algorithm to dataset

In [142]:
# import data
import pandas as pd
import os
current_directory = os.getcwd()
seeds = pd.read_csv(os.path.join(current_directory,'seeds.csv'), header=0)
seeds_features = seeds.iloc[:, :-1]
seeds_true_label = seeds.iloc[:, -1]

vowel = pd.read_csv(os.path.join(current_directory,'Vowel.csv'), header=0)
vowel_features = vowel.iloc[:, :-1]
vowel_true_label = vowel.iloc[:, -1]

### 3.1 seeds: Clustering algorithm & evaluation metrics

In [143]:
for i in [2,3]:
    # K-means
    kmeans_seeds = KMeans(n_clusters=i)
    kmeans_seeds.fit(seeds_features.values)

    silhouette_seeds_kmeans = silhouette_coefficient(seeds_features.values, kmeans_seeds.labels)
    rand_index_seeds_kmeans = rand_index(seeds_true_label.values, kmeans_seeds.labels)
    nmi_seeds_kmeans = NMI(seeds_true_label.values, kmeans_seeds.labels)
    
    # GMM
    gmm_seeds = GMM(n_components=i)
    gmm_seeds.fit(seeds_features.values)

    silhouette_seeds_gmm = silhouette_coefficient(seeds_features.values, gmm_seeds.labels)
    rand_index_seeds_gmm = rand_index(seeds_true_label.values, gmm_seeds.labels)
    nmi_seeds_gmm = NMI(seeds_true_label.values, gmm_seeds.labels)
    
    # print results
    print("\nResults for seeds dataset:")
    print("Number of clusters (k):", i)
    print("K-means Silhouette Coefficient:", silhouette_seeds_kmeans)
    print("K-means Rand Index:", rand_index_seeds_kmeans)
    print("K-means NMI:", nmi_seeds_kmeans)
    print("GMM Silhouette Coefficient:", silhouette_seeds_gmm)
    print("GMM Rand Index:", rand_index_seeds_gmm)
    print("GMM NMI:", nmi_seeds_gmm)


Results for seeds dataset:
Number of clusters (k): 2
K-means Silhouette Coefficient: 0.5228955002005705
K-means Rand Index: 0.7309637730690363
K-means NMI: 0.7074384585185741
GMM Silhouette Coefficient: 0.45861106387924744
GMM Rand Index: 0.73734335839599
GMM NMI: 0.5926542996750351

Results for seeds dataset:
Number of clusters (k): 3
K-means Silhouette Coefficient: 0.47943599429227485
K-means Rand Index: 0.8743677375256322
K-means NMI: 0.8205373603161036
GMM Silhouette Coefficient: 0.38082934469644125
GMM Rand Index: 0.8621553884711779
GMM NMI: 0.8500556625658585


### 3.2 Vowel: Clustering algorithm & evaluation metrics

In [157]:
'''The class in the dataset Vowel is written in string, so we need to convert it into integer.'''
def label_encoder(labels):   
    unique_labels = np.unique(labels)
    label_dict = {label: idx for idx, label in enumerate(unique_labels)}
    return np.array([label_dict[label] for label in labels])

for i in range(2,12):
    # K-means
    kmeans_vowel = KMeans(n_clusters=i)
    kmeans_vowel.fit(vowel_features.values)
    vowel_true_label_encoded = label_encoder(vowel_true_label.values)  # Convert string labels to integers

    silhouette_vowel_kmeans = silhouette_coefficient(vowel_features.values, kmeans_vowel.labels)
    rand_index_vowel_kmeans = rand_index(vowel_true_label.values, kmeans_vowel.labels)
    nmi_vowel_kmeans = NMI(vowel_true_label_encoded, kmeans_vowel.labels)
    
    # GMM
    gmm_vowel = GMM(n_components=i)
    gmm_vowel.fit(vowel_features.values)
    vowel_true_label_encoded = label_encoder(vowel_true_label.values)  # Convert string labels to integers

    silhouette_vowel_gmm = silhouette_coefficient(vowel_features.values, gmm_vowel.labels)
    rand_index_vowel_gmm = rand_index(vowel_true_label.values, gmm_vowel.labels)
    nmi_vowel_gmm = NMI(vowel_true_label_encoded, gmm_vowel.labels)
    
    # print results
    print("\nResults for Vowel dataset:")
    print("Number of clusters (k):", i)
    print("K-means Silhouette Coefficient:", silhouette_vowel_kmeans)
    print("K-means Rand Index:", rand_index_vowel_kmeans)
    print("K-means NMI:", nmi_vowel_kmeans)
    print("GMM Silhouette Coefficient:", silhouette_vowel_gmm)
    print("GMM Rand Index:", rand_index_vowel_gmm)
    print("GMM NMI:", nmi_vowel_gmm)


Results for Vowel dataset:
Number of clusters (k): 2
K-means Silhouette Coefficient: 0.4915868678690223
K-means Rand Index: 0.49767441860465117
K-means NMI: 0.39392317145655925
GMM Silhouette Coefficient: 0.06372575009670514
GMM Rand Index: 0.508604753296361
GMM NMI: 0.3958554866794623

Results for Vowel dataset:
Number of clusters (k): 3
K-means Silhouette Coefficient: 0.37996891703580543
K-means Rand Index: 0.6287158746208291
K-means NMI: 0.40813945125756407
GMM Silhouette Coefficient: -0.03528727304452447
GMM Rand Index: 0.6347764806814352
GMM NMI: 0.41049244486503367

Results for Vowel dataset:
Number of clusters (k): 4
K-means Silhouette Coefficient: 0.3113853454986848
K-means Rand Index: 0.6942366026289181
K-means NMI: 0.4340348911844581
GMM Silhouette Coefficient: 0.008319179391817604
GMM Rand Index: 0.7199701769974773
GMM NMI: 0.4340806838781665

Results for Vowel dataset:
Number of clusters (k): 5
K-means Silhouette Coefficient: 0.2893539431868894
K-means Rand Index: 0.738783